In [3]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [49]:
# 1. 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv') 

C:\Users\bonomania\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [48]:
# 2. 영화정보 중 id만 가져오기
tmdb_movie_list.info()
tmdb_movie_id_list = tmdb_movie_list[["id"]]
# del tmdb_movie_list

NameError: name 'tmdb_movie_list' is not defined

In [39]:
print(tmdb_movie_id_list.shape)
tmdb_movie_id_list.head()

(630186, 1)


,id
0,2.0
1,3.0
2,5.0
3,6.0
4,8.0


In [40]:
## 영화 cast와 crew정보 3)영화id를 이용하여 cast와 crew정보 crawling  (너무 오래걸림)

all_crew_list = []
all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):
    try :
        cast_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(cast_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        # cast_list += movie_data["credit"]["cast"]
        cast_list = movie_data["credits"]["cast"]
        for cast in cast_list:
            cast["movie_id"] = movie_id
            all_cast_list.append(cast)
        # crew_list += movie_data["credit"]['crew']
        crew_list = movie_data["credits"]["crew"]
        for crew in crew_list:
            crew["movie_id"] = movie_id
            all_crew_list.append(crew)
    except:
        error_list.append(movie_id)

In [ ]:
tmdb_movie_id_list["id"].values

In [31]:
# crew 정보 csv파일로 저장 
crews=pd.DataFrame(all_crew_list)
crews
# crews.to_csv("crews.csv", mode='w')

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,credit_id,department,job,movie_id
0,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.62,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,52fe420dc3a36847f800001f,Writing,Screenplay,2.0
1,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.62,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,52fe420dc3a36847f8000025,Directing,Director,2.0
2,False,2,16767,Directing,Aki Kaurismäki,Aki Kaurismäki,1.62,/t0pdsYfLts7p01OBN1P9UHp1Z0l.jpg,5be040dac3a3682b34043d42,Production,Producer,2.0
3,False,2,16769,Camera,Timo Salminen,Timo Salminen,0.98,/a8rH6eJ0XY1zQph18eNDj7MC8y4.jpg,52fe420dc3a36847f800003b,Camera,Director of Photography,2.0
4,False,1,54766,Editing,Raija Talvio,Raija Talvio,1.38,None,52fe420dc3a36847f8000041,Editing,Editor,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
178,False,0,891,Sound,Moritz Hoffmeister,Moritz Hoffmeister,0.60,None,52fe420dc3a36847f800032f,Sound,Sound Designer,9.0
179,False,1,914,Directing,Anna Haas,Anna Haas,0.60,None,52fe420dc3a36847f800033d,Directing,Director,9.0
180,False,0,917,Camera,Jesko Klatt,Jesko Klatt,0.60,None,52fe420dc3a36847f8000343,Camera,Underwater Camera,9.0
181,False,0,919,Visual Effects,Jan Raiber,Jan Raiber,0.60,None,52fe420dc3a36847f8000349,Visual Effects,Animation,9.0


In [ ]:
# crew 정보 DB(mysql)에 저장 (테이블 만들어야 함)

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

with conn.cursor() as cursor: # with가 종료될 때 cursor.close() 자동 호출
    cursor.execute()
    cursor.execute()
    conn.commit() # 이전에 실행된 SQL 결과를 확정

conn.close()

In [32]:
# cast 정보 csv파일로 저장 
casts=pd.DataFrame(all_cast_list)
casts.to_csv("crews.csv", mode='w')

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,movie_id
0,False,2,54768,Acting,Turo Pajala,Turo Pajala,1.725,/b6JdzqTn6UFYf4DouHbbE8Ypk4r.jpg,3,Taisto Olavi Kasurinen,52fe420dc3a36847f8000029,0,2.0
1,False,1,54769,Acting,Susanna Haavisto,Susanna Haavisto,1.289,/lhjoqlBFCvshchMAd2lQLWV7z7I.jpg,4,Irmeli Katariina Pihlaja,52fe420dc3a36847f800002d,1,2.0
2,False,2,4826,Acting,Matti Pellonpää,Matti Pellonpää,1.607,/1Qzhkkp3wFE2NMGKVyOra9931q2.jpg,5,Mikkonen,52fe420dc3a36847f8000031,2,2.0
3,False,2,54770,Acting,Eetu Hilkamo,Eetu Hilkamo,0.600,None,6,Riku,52fe420dc3a36847f8000035,3,2.0
4,False,2,1177496,Acting,Erkki Pajala,Erkki Pajala,0.766,/lJ7GmiHT1UPxmURqQCLT4L5PIMa.jpg,12,Miner,5e1cda97d5dbc20017f439bb,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,False,1,105988,Acting,Galyn Görg,Galyn Görg,2.451,/sr3PUWk3Qo0b6cYh3pdSloBAgbx.jpg,20,Clarissa,52fe420dc3a36847f8000289,12,6.0
84,False,1,22621,Acting,Angela Alvarado,Angela Alvarado,1.443,None,21,Rita,52fe420dc3a36847f800028d,13,6.0
85,False,0,1226696,Acting,Will Zahrn,Will Zahrn,0.600,/5JDnuufySepQ1fig7Hga4xL4nio.jpg,34,Charley,560c6b95c3a368681f0064e8,14,6.0
86,False,1,901,Acting,Rita Lengyel,Rita Lengyel,1.048,None,7,Alexandra,52fe420dc3a36847f8000333,0,9.0
